# BASIC CRUD OPERATIONS

In [32]:
import pandas as pd
from datetime import datetime
from pymongo import MongoClient
# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

In [33]:
print("Collections in eduhub:")
print(db.list_collection_names())

Collections in eduhub:
['users', 'assignments', 'enrollments', 'courses', 'submissions', 'lessons']


## CREATE OPERATIONS

In [34]:
# Adding a new student
new_student = {
    "userId": "STU020",
    "email": "student17@example.com",
    "firstName": "Ada",
    "lastName": "Eze",
    "role": "student",
    "dateJoined": datetime.now(),
    "profile": {
        "bio": "Passionate learner exploring data science and analytics.",
        "avatar": "https://example.com/avatar16.png",
        "skills": ["Python", "SQL"]
    },
    "isActive": True
}

try:
    db.users.insert_one(new_student)
    print("Added new student")
except Exception as e:
    print("Error adding new student:", e)


# Creating a new course
new_course = {
    "courseId": "CRS012",
    "title": "Data Visualization with Power BI",
    "description": "Learn to design and develop interactive dashboards with Power BI.",
    "instructorId": "INST001",
    "category": "Data Science",
    "level": "beginner",
    "duration": 40,
    "price": 99.99,
    "tags": ["data", "visualization", "business-intelligence"],
    "createdAt": datetime.now(),
    "updatedAt": datetime.now(),
    "isPublished": True
}

try:
    db.courses.insert_one(new_course)
    print("Added new course")
except Exception as e:
    print("Error adding new course:", e)


# Enrolling a student in a course
new_enrollment = {
    "enrollmentId": "ENR019",
    "studentId": "STU016",
    "courseId": "CRS009",
    "enrolledAt": datetime.now(),
    "status": "active"
}

try:
    db.enrollments.insert_one(new_enrollment)
    print("Enrolled student in course")
except Exception as e:
    print("Error enrolling student:", e)


# Adding a new lesson to an existing course
new_lesson = {
    "lessonId": "LES030",
    "courseId": "CRS009",
    "title": "Getting Started with Power BI",
    "content": "This lesson introduces the Power BI interface and its core features.",
    "order": 1,
    "resources": ["https://docs.microsoft.com/en-us/power-bi/"],
    "createdAt": datetime.now(),
    "updatedAt": datetime.now()
}

try:
    db.lessons.insert_one(new_lesson)
    print("Added new lesson to course")
except Exception as e:
    print("Error adding new lesson:", e)

Added new student
Added new course
Enrolled student in course
Added new lesson to course


## READ OPERATIONS


### Active Students 

In [ ]:
#Finding all active student

try:
    # Fetch active students
    active_students = db.users.find(
        {"role": "student", "isActive": True},
        {"_id": 0, "userId": 1, "firstName": 1, "lastName": 1, "email": 1}
    )
    active_students_list = list(active_students)

    # Convert to DataFrame
    df_active_students = pd.DataFrame(active_students_list)

    # Reorder columns
    df_active_students = df_active_students[['userId', 'firstName', 'lastName', 'email']]

    # Display the DataFrame
    print("Active Students:\n")
    print(df_active_students)

except Exception as e:
    print("Error fetching active students:", e)

Active Students:

   userId firstName    lastName                    email
0  STU001      Alan       Brown  tammyashley@example.net
1  STU003   Michael       Davis     jerome18@example.net
2  STU005     Jacob     Johnson   mblanchard@example.net
3  STU009     Harry       Smith      frivera@example.org
4  STU010   Richard       Klein    tfriedman@example.com
5  STU011      Leah       Clark     paulcruz@example.com
6  STU013     Angel       Clark       smccoy@example.net
7  STU014   Shannon  Villanueva  josephbrown@example.com
8  STU020       Ada         Eze    student17@example.com


### Retrieving Course Details with Instructor Information

In [ ]:
# Retrieving the Course Details 
try:
    course_with_instructor = db.courses.aggregate([
        {
            "$lookup": {
                "from": "users",
                "localField": "instructorId",
                "foreignField": "userId",
                "as": "instructor"
            }
        },
        { "$unwind": "$instructor" },
        {
            "$project": {
                "_id": 0,
                "courseId": 1,
                "title": 1,
                "category": 1,
                "level": 1,
                "instructorFirstName": "$instructor.firstName",
                "instructorLastName": "$instructor.lastName",
                "instructorEmail": "$instructor.email"
            }
        }
    ])
    
    courses_list = list(course_with_instructor)

    # Convert to DataFrame
    df_courses = pd.DataFrame(courses_list)

    # Optional: reorder columns
    df_courses = df_courses[['courseId', 'title', 'category', 'level','instructorFirstName', 'instructorLastName', 'instructorEmail']]

    print("Courses with Instructor Info:\n")
    print(df_courses)

except Exception as e:
    print("Error fetching courses with instructor info:", e)

Courses with Instructor Info:

  courseId                             title         category         level  \
0   CRS001   Frontend Development with React  Web Development  intermediate   
1   CRS002          Cloud Computing with AWS  Cloud Computing  intermediate   
2   CRS003        Full-Stack Web Development  Web Development      advanced   
3   CRS004  Serverless Applications on Cloud  Cloud Computing      beginner   
4   CRS005      Introduction to Data Science     Data Science      beginner   
5   CRS006   Frontend Development with React  Web Development      advanced   
6   CRS007    Machine Learning for Beginners     Data Science      advanced   
7   CRS008  Backend Development with Node.js  Web Development      advanced   
8   CRS012  Data Visualization with Power BI     Data Science      beginner   

  instructorFirstName instructorLastName            instructorEmail  
0               Terri               Reid         chad43@example.com  
1               Emily          Hernand

###  Getting all Courses in a Specific Category

In [47]:
#Getting all courses in a specific category
category = "Data Science"

try:
    # Query courses in the category
    courses_in_category = db.courses.find(
        {"category": category},
        {"_id": 0, "courseId": 1, "title": 1, "level": 1}
    )
    
    # Convert courses to list
    courses_list = list(courses_in_category)
    
    # Convert list to DataFrame
    df_courses_category = pd.DataFrame(courses_list)
    
    # Reorder columns
    df_courses_category = df_courses_category[['courseId', 'title', 'level']]
    
    # Display DataFrame
    print(f"Courses in {category}:\n")
    print(df_courses_category)

except Exception as e:
    print(f"Error fetching courses in category '{category}': {e}")

Courses in Data Science:

  courseId                             title     level
0   CRS005      Introduction to Data Science  beginner
1   CRS007    Machine Learning for Beginners  advanced
2   CRS012  Data Visualization with Power BI  beginner


### Finding Students Enrolled in a Particular Course


In [ ]:
# Finding student enrolled in a particular course
course_id = "CRS002"

try:
    students_in_course = db.enrollments.aggregate([
        { "$match": {"courseId": course_id} },
        {
            "$lookup": {
                "from": "users",
                "localField": "studentId",
                "foreignField": "userId",
                "as": "student"
            }
        },
        { "$unwind": "$student" },
        {
            "$project": {
                "_id": 0,
                "userId": "$student.userId",
                "firstName": "$student.firstName",
                "lastName": "$student.lastName",
                "email": "$student.email"
            }
        }
    ])

    # Convert aggregation cursor to list
    students_list = list(students_in_course)

    # Convert list to DataFrame
    df_students = pd.DataFrame(students_list)

    # Reorder columns
    df_students = df_students[['userId', 'firstName', 'lastName', 'email']]

    # Display DataFrame
    print(f"Students enrolled in {course_id}:\n")
    print(df_students)

except Exception as e:
    print(f"Error fetching students for course '{course_id}': {e}")

Students enrolled in CRS002:

   userId firstName lastName                    email
0  STU006      Lisa   Turner   daviserica@example.com
1  STU012    Daniel  Gilbert  harryfoster@example.com
2  STU003   Michael    Davis     jerome18@example.net


### Searching Courses by Title (case-insensitive, partial match)

In [ ]:
# Searching courses by Title
search_term = "Web Development"

try:
    # Query courses matching the regex
    matched_courses = db.courses.find(
        {"title": {"$regex": search_term, "$options": "i"}},
        {"_id": 0, "courseId": 1, "title": 1, "category": 1, "level": 1}
    )

    matched_courses_list = list(matched_courses)

    # Convert list to DataFrame
    df_matched_courses = pd.DataFrame(matched_courses_list)

    # Reorder columns
    df_matched_courses = df_matched_courses[['courseId', 'title', 'category', 'level']]

    # Display DataFrame
    print(f"Courses matching '{search_term}':\n")
    print(df_matched_courses)

except Exception as e:
    print(f"Error searching courses with term '{search_term}': {e}")

Courses matching 'Web Development':

  courseId                       title         category     level
0   CRS003  Full-Stack Web Development  Web Development  advanced


## UPDATE OPERATIONS

### Update a User’s Profile Information

In [54]:
#Updating a Useers Profile information
user_id = "STU005"

try:
    db.users.update_one(
        {"userId": user_id},
        {
            "$set": {
                "profile.bio": "Enthusiastic learner with interest in AI and Machine Learning.",
                "profile.avatar": "https://example.com/new_avatar.png",
                "profile.skills": ["Python", "SQL", "Machine Learning"]
            }
        }
    )
    print(f"User {user_id} profile updated successfully.")

except Exception as e:
    print(f"Error updating user {user_id}: {e}")

User STU005 profile updated successfully.


### Mark a Course as Published

In [55]:
course_id = "CRS003"

try:
    db.courses.update_one(
        {"courseId": course_id},
        {"$set": {"isPublished": True, "updatedAt": datetime.now()}}
    )
    print(f"Course {course_id} marked as published successfully.")

except Exception as e:
    print(f"Error marking course {course_id} as published: {e}")

Course CRS003 marked as published successfully.


### Update Assignment Grades

In [56]:
#Updating Assignment Grades
submission_id = "SUB010"

try:
    db.submissions.update_one(
        {"submissionId": submission_id},
        {
            "$set": {
                "grade": 80,
                "feedback": "Excellent work, keep it up!",
                "gradedAt": datetime.now()
            }
        }
    )
    print(f"Updated grade for submission {submission_id} successfully.")

except Exception as e:
    print(f"Error updating grade for submission {submission_id}: {e}")

Updated grade for submission SUB010 successfully.


### Add Tags to an Existing Course

In [57]:
#Adding Tags to an Existing Course
course_id = "CRS005"

try:
    db.courses.update_one(
        {"courseId": course_id},
        {"$addToSet": {"tags": {"$each": ["AI", "Data Science", "Python"]}}}
    )
    print(f"Tags updated for course {course_id} successfully.")

except Exception as e:
    print(f"Error updating tags for course {course_id}: {e}")

Tags updated for course CRS005 successfully.


## DELETE OPERATIONS

### Remove a User (soft delete by setting isActive to false)


In [58]:
# Removing user
user_id = "STU006"

try:
    db.users.update_one(
        {"userId": user_id},
        {"$set": {"isActive": False, "updatedAt": datetime.now()}}
    )
    print(f"User {user_id} has been deactivated successfully.")
except Exception as e:
    print(f"Error deactivating user {user_id}: {e}")


User STU006 has been deactivated successfully.


### Deleting  an Enrollment

In [59]:
#Deleting an Enrollment
enrollment_id = "ENR005"

try:
    delete_result = db.enrollments.delete_one({"enrollmentId": enrollment_id})
    print(f"Deleted {delete_result.deleted_count} enrollment(s) with ID {enrollment_id}")
except Exception as e:
    print(f"Error deleting enrollment {enrollment_id}: {e}")

Deleted 1 enrollment(s) with ID ENR005


###  Remove a Lesson from a Course

In [60]:
# Removing a lesson from a Course
lesson_id = "LES008"

try:
    delete_result = db.lessons.delete_one({"lessonId": lesson_id})
    print(f"Deleted {delete_result.deleted_count} lesson(s) with ID {lesson_id}")
except Exception as e:
    print(f"Error deleting lesson {lesson_id}: {e}")

Deleted 1 lesson(s) with ID LES008
